In [1]:
import pandas as pd

df_model_data_CAN = pd.read_csv('../data/processed/CAN_Categorized_Products_and_Services.csv')
df_model_data_USA = pd.read_csv('../data/processed/US_Categorized_Products_and_Services.csv')
df_Canada_CPI = pd.read_csv('../data/processed/Canada_CPI_Processed_2018_2019.csv')
df_USA_CPI = pd.read_csv('../data/processed/USA_CPI_Processed_2018_2019.csv')

In [2]:
from sklearn.preprocessing import StandardScaler

df_USA_CPI_melt = pd.melt(df_USA_CPI, var_name = 'Products and product groups',value_name = 'VALUE', id_vars = 'REF_DATE')

df_Canada_CPI_Scaled = df_Canada_CPI.copy()
df_Canada_CPI_NonScaled = df_Canada_CPI.copy()

df_USA_CPI_Scaled = df_USA_CPI_melt.copy()
df_USA_CPI_NonScaled = df_USA_CPI_melt.copy()
for unique_product in df_Canada_CPI_Scaled['Products and product groups'].unique():
    stan_scale = StandardScaler()
    mask = df_Canada_CPI_Scaled['Products and product groups']==unique_product
    df_Canada_CPI_Scaled.loc[mask, "VALUE"] = stan_scale.fit_transform(df_Canada_CPI_Scaled.loc[mask, "VALUE"].values[:, None]).flatten()
for unique_product in df_USA_CPI_Scaled['Products and product groups'].unique():
    stan_scale = StandardScaler()
    mask = df_USA_CPI_Scaled['Products and product groups']==unique_product
    df_USA_CPI_Scaled.loc[mask, "VALUE"] = stan_scale.fit_transform(df_USA_CPI_Scaled.loc[mask, "VALUE"].values[:, None]).flatten()

In [3]:
dict_CAN = df_model_data_CAN.drop_duplicates(subset=['Product_Service']).set_index('Product_Service')['Category'].to_dict()
dict_USA = df_model_data_USA.drop_duplicates(subset=['Product_Service']).set_index('Product_Service')['Category'].to_dict()

df_Canada_CPI_Scaled['Category'] = df_Canada_CPI['Products and product groups'].map(dict_CAN)
df_Canada_CPI_NonScaled['Category'] = df_Canada_CPI['Products and product groups'].map(dict_CAN)
df_USA_CPI_Scaled['Category'] = df_USA_CPI_Scaled['Products and product groups'].map(dict_USA)
df_USA_CPI_NonScaled['Category'] = df_USA_CPI_NonScaled['Products and product groups'].map(dict_USA)


In [4]:
#make groupby dataframe
df_Canada_CPI_Cat = df_Canada_CPI_Scaled.copy().drop(columns = ['Products and product groups'])
df_Canada_CPI_groupby = df_Canada_CPI_Cat.groupby(['REF_DATE', 'Category']).median().reset_index()

df_Canada_CPI_Cat_NonScaled = df_Canada_CPI_NonScaled.copy().drop(columns = ['Products and product groups'])
df_Canada_CPI_groupby_NonScaled = df_Canada_CPI_Cat_NonScaled.groupby(['REF_DATE', 'Category']).median().reset_index()

df_USA_CPI_Cat = df_USA_CPI_Scaled.copy().drop(columns = ['Products and product groups'])
df_USA_CPI_groupby = df_USA_CPI_Cat.groupby(['REF_DATE', 'Category']).median().reset_index()

df_USA_CPI_Cat_NonScaled = df_USA_CPI_NonScaled.copy().drop(columns = ['Products and product groups'])
df_USA_CPI_groupby_NonScaled = df_USA_CPI_Cat_NonScaled.groupby(['REF_DATE', 'Category']).median().reset_index()

In [5]:
for column in df_Canada_CPI_groupby['Category'].unique():
    print(column)

???
Alcohol & Tobacco
Appliances & Equipment
Clothing & Footwear
Communication
Education & Reading
Energy
Furniture & Home Decor
Gardening
Groceries
Health & Personal Care
Household Supplies
Housing & Utilities
Housing, Rent, Tax  & Insurance
Recreation & Entertainment
Services
Transportation
Utilities
Vehicles


In [6]:
import statsmodels.formula.api as smf
import numpy as np
import altair as alt

def plot_structure(df, category, date1, date2, feature = 'Category', goodtype = None):
    print("Product: ", category)
    df_in_question = df.copy()
    if goodtype == None:
        df_in_question = df_in_question[df_in_question[feature] == category]
    else:
        df_in_question = df_in_question[(df_in_question[feature] == category) & (df_in_question['GoodType'] == goodtype)]
    df_in_question = df_in_question[(df_in_question['REF_DATE']>=date1) & (df_in_question['REF_DATE']<=date2)]
    df_in_question = df_in_question.sort_values('REF_DATE')
    chart = alt.Chart(df_in_question).mark_line().encode(
        x='REF_DATE',
        y='VALUE',
        color=feature
    )
    display(chart)
    return None

def regression_discontinuity_model(df, category, date1, date2, date3, feature = 'Category', goodtype = None):
    print("Product: ", category)
    df_in_question = df.copy()
    if goodtype ==None: 
        df_in_question = df_in_question[df_in_question[feature] == category]
    else:
        df_in_question = df_in_question[(df_in_question[feature] == category) & (df_in_question['GoodType'] == goodtype)]
    
    df_in_question = df_in_question[(df_in_question['REF_DATE']>=date1) & (df_in_question['REF_DATE']<=date2)]
    df_in_question = df_in_question.sort_values('REF_DATE')
    
    df_in_question['VALUE_DIFF'] = df_in_question['VALUE'].diff().shift(-1)
    df_in_question = df_in_question.iloc[:-1]
    df_in_question.dropna(subset=['VALUE_DIFF'], inplace=True)
    
    chart = alt.Chart(df_in_question).mark_line().encode(
        x='REF_DATE',
        y='VALUE_DIFF',
        color=feature
    )
    chart2 = alt.Chart(df_in_question).mark_line().encode(
        x='REF_DATE',
        y='VALUE',
        color=feature
    )
    display(chart2)
    display(chart)
    
    df_in_question.dropna(inplace=True)
    
    df_in_question['treatment'] = np.where(df_in_question['REF_DATE']>=date3, 1, 0)
    df_in_question = df_in_question.sort_values(by=['Category', 'REF_DATE'])
    # df_in_question["VALUE"] = df_in_question["VALUE"].rolling(window=3).mean()
    df_in_question.dropna(inplace=True)
    dict_replace = {unique_date: count for count, unique_date in enumerate(df_in_question['REF_DATE'].unique())}
    
    df_in_question['Date_Replaced'] = df_in_question['REF_DATE'].map(dict_replace)
    
    for date, num_date in zip(df_in_question['REF_DATE'].iloc[:], df_in_question['Date_Replaced'].iloc[:]):
        
        if date == date3:
            final_date = num_date
            
    df_in_question['Num_Date'] = df_in_question['Date_Replaced'] - final_date
    
    df_in_question['Num_Date'] = df_in_question['Num_Date'].astype(float)
    model = smf.ols(formula = 'VALUE ~ Num_Date + treatment + Num_Date:treatment', data=df_in_question).fit()
    
    return model

def differences_differences(df, category1, category2, date1, date2, date3, date4=None, feature='Category'):
    df_in_question = df.copy()
    
    
    df_differences_differences = df_in_question[(df_in_question[feature]==category1) | (df_in_question[feature]==category2)]
    df_differences_differences = df_differences_differences[(df_differences_differences['REF_DATE']<=date2)&(df_differences_differences['REF_DATE']>=date1)]
    scale = StandardScaler()
        
    df_differences_differences = df_differences_differences[(df_differences_differences[feature]==category1)|(df_differences_differences[feature]==category2)]
    
    if date4==None:
        pass
    else:
        mask = (df_differences_differences['REF_DATE']<=date4)&(df_differences_differences['REF_DATE']>=date3)
        mask2 = df_differences_differences['REF_DATE']==date3
        new_value = df_differences_differences[mask]['VALUE'].median()
        df_differences_differences.loc[mask2, "VALUE"] = new_value
        df_differences_differences = df_differences_differences[(df_differences_differences['REF_DATE']<=date3) | (df_differences_differences['REF_DATE']>date4)]

    df_differences_differences['VALUE'] = scale.fit_transform(df_differences_differences['VALUE'].values.reshape(-1, 1))
    
    
    df_differences_differences['post'] = np.where(df_differences_differences['REF_DATE']>date3, 1, 0)
    df_differences_differences['tariff_non_tariffed'] = np.where(df_differences_differences['Category']==category1, 1, 0)
    
    model = smf.ols(formula = 'VALUE ~  tariff_non_tariffed + post + tariff_non_tariffed:post', data=df_differences_differences).fit().summary()
    
    
    return model



In [7]:
model1 = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Groceries', '2018-05-01', '2018-09-01', '2018-08-01')



Product:  Groceries


alt.Chart(...)

alt.Chart(...)

In [8]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     3.822
Date:                Wed, 12 Mar 2025   Prob (F-statistic):             0.0102
Time:                        19:35:53   Log-Likelihood:                -399.40
No. Observations:                 354   AIC:                             806.8
Df Residuals:                     350   BIC:                             822.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.0910      0.136     -0.667      0.505      -0.359       0.177
Num_Date               0.1492      0.063      2.364      0.019       0.025       0.273
treatment             -0.0774      0.163     -0.475      0.635      -0.398       0.243
Num_Date:treatment    -0.4286      0.142     -3.029      0.003      -0.707      -0.150
==============================================================================
Omnibus:                        6.811   Durbin-Watson:                   2.188
Prob(Omnibus):                  0.033   Jarque-Bera (JB):                6.603
Skew:                           0.314   Prob(JB):                       0.0368
Kurtosis:                       3.231   Cond. No.                         9.94
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
#test parallel trends assumption:
def plot_for_parallel_trends(df, date1, date2, category_tariff, category_non_tariff, category_3 = None):
    df_Canada_CPI_Scaled_US_on_Canada_Tariffs = df[(df['REF_DATE']<=date2)&(df['REF_DATE']>=date1)]
    scale = StandardScaler()
    if category_3 ==None:
        df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1 = df_Canada_CPI_Scaled_US_on_Canada_Tariffs[(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_tariff)|(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_non_tariff)]
    else:
        df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1 = df_Canada_CPI_Scaled_US_on_Canada_Tariffs[(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_tariff)|(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_non_tariff)|(df_Canada_CPI_Scaled_US_on_Canada_Tariffs['Category']==category_3)]
    df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))
    chart1 = alt.Chart(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1).mark_line().encode(
        x='REF_DATE',
        y='VALUE',
        color = 'Category'
    )

    return chart1

In [10]:
plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-06-01', '2018-09-01', 'Groceries', 'Housing, Rent, Tax  & Insurance')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [11]:
print(differences_differences(df_Canada_CPI_NonScaled, 'Groceries', 'Housing, Rent, Tax  & Insurance', '2018-06-01', '2018-09-01', '2018-08-01'))
differences_differences(df_Canada_CPI_groupby_NonScaled, 'Groceries', 'Housing, Rent, Tax  & Insurance', '2018-06-01', '2018-09-01', '2018-08-01')

                            OLS Regression Results                            
Dep. Variable:                  VALUE   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     3.482
Date:                Wed, 12 Mar 2025   Prob (F-statistic):             0.0162
Time:                        19:35:53   Log-Likelihood:                -454.53
No. Observations:                 324   AIC:                             917.1
Df Residuals:                     320   BIC:                             932.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     25.46
Date:                Wed, 12 Mar 2025   Prob (F-statistic):            0.00456
Time:                        19:35:53   Log-Likelihood:                0.65074
No. Observations:                   8   AIC:                             6.699
Df Residuals:                       4   BIC:                             7.016
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.9338      0.182      5.127      0.007       0.428       1.439
tariff_non_tariffed         -1.8264      0.258     -7.091      0.002      -2.542      -1.111
post                         0.1421      0.364      0.390      0.716      -0.869       1.154
tariff_non_tariffed:post    -0.4487      0.515     -0.871      0.433      -1.879       0.982
==============================================================================
Omnibus:                        1.424   Durbin-Watson:                   0.699
Prob(Omnibus):                  0.491   Jarque-Bera (JB):                0.574
Skew:                          -0.628   Prob(JB):                        0.750
Kurtosis:                       2.616   Cond. No.                         6.50
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
#It seems like the groupby shows some interaction while the grouping of all items in grocerys does not
#let us investigate into specific grocery categories to see which groceries may have been impacted by the tariffs

In [13]:

grocery_items = df_Canada_CPI_NonScaled[(df_Canada_CPI_NonScaled['Category'] == 'Groceries')]['Products and product groups'].unique()

grocery_dataframe = df_Canada_CPI_NonScaled[(df_Canada_CPI_NonScaled['Category'] == 'Groceries') | (df_Canada_CPI_NonScaled['Category'] == 'Housing, Rent, Tax  & Insurance')]


dict_grocery = food_categories = {
    'Apples': 'Fruit',
    'Baby foods': 'Prepared Foods',
    'Bakery and cereal products (excluding baby food)': 'Bakery and Cereal',
    'Bakery products': 'Bakery and Cereal',
    'Bananas': 'Fruit',
    'Bread, rolls and buns': 'Bakery and Cereal',
    'Breakfast cereal and other cereal products (excluding baby food)': 'Bakery and Cereal',
    'Butter': 'Dairy',
    'Canned and other preserved fish': 'Fish and Seafood',
    'Canned vegetables and other vegetable preparations': 'Vegetables',
    'Cereal products (excluding baby food)': 'Bakery and Cereal',
    'Cheese': 'Dairy',
    'Coffee': 'Beverages',
    'Coffee and tea': 'Beverages',
    'Condiments, spices and vinegars': 'Condiments and Spices',
    'Confectionery': 'Sweets',
    'Cookies and crackers': 'Bakery and Cereal',
    'Dairy products': 'Dairy',
    'Dairy products and eggs': 'Dairy',
    'Edible fats and oils': 'Fats and Oils',
    'Eggs': 'Dairy',
    'Fish': 'Fish and Seafood',
    'Fish, seafood and other marine products': 'Fish and Seafood',
    'Flour and flour-based mixes': 'Bakery and Cereal',
    'Food': 'General Category',
    'Food and energy': 'General Category',
    'Food purchased from cafeterias and other restaurants': 'Dining Out',
    'Food purchased from fast food and take-out restaurants': 'Dining Out',
    'Food purchased from restaurants': 'Dining Out',
    'Food purchased from stores': 'General Category',
    'Food purchased from table-service restaurants': 'Dining Out',
    'Fresh fruit': 'Fruit',
    'Fresh fruit and vegetables': 'Produce',
    'Fresh milk': 'Dairy',
    'Fresh or frozen beef': 'Meat',
    'Fresh or frozen chicken': 'Meat',
    'Fresh or frozen fish (including portions and fish sticks)': 'Fish and Seafood',
    'Fresh or frozen meat (excluding poultry)': 'Meat',
    'Fresh or frozen pork': 'Meat',
    'Fresh or frozen poultry': 'Meat',
    'Fresh vegetables': 'Vegetables',
    'Frozen and dried vegetables': 'Vegetables',
    'Frozen food preparations': 'Prepared Foods',
    'Fruit juices': 'Beverages',
    'Fruit, fruit preparations and nuts': 'Fruit',
    'Goods excluding food purchased from stores': 'General Category',
    'Goods excluding food purchased from stores and energy': 'General Category',
    'Ham and bacon': 'Meat',
    'Ice cream and related products': 'Dairy',
    'Lettuce': 'Vegetables',
    'Margarine': 'Fats and Oils',
    'Meat': 'Meat',
    'Non-alcoholic beverages': 'Beverages',
    'Non-durable goods excluding food purchased from stores': 'General Category',
    'Non-durable goods excluding food purchased from stores and energy': 'General Category',
    'Nuts and seeds': 'Nuts and Seeds',
    'Oranges': 'Fruit',
    'Pasta products': 'Pasta',
    'Pet food and supplies': 'Pet Supplies',
    'Potatoes': 'Vegetables',
    'Preserved fruit and fruit preparations': 'Fruit',
    'Preserved vegetables and vegetable preparations': 'Vegetables',
    'Processed meat': 'Meat',
    'Rice and rice-based mixes': 'Grains',
    'Seafood and other marine products': 'Fish and Seafood',
    'Soup': 'Prepared Foods',
    'Sugar and confectionery': 'Sweets',
    'Sugar and syrup': 'Sweets',
    'Tea': 'Beverages',
    'Tomatoes': 'Vegetables',
    'Vegetables and vegetable preparations': 'Vegetables',
    'Homeowners': 'Housing, Rent, Tax  & Insurance',
    'Homeowners\' replacement cost': 'Housing, Rent, Tax  & Insurance',
    'Homeowners\' maintenance and repairs': 'Housing, Rent, Tax  & Insurance',
    'Mortgage interest cost': 'Housing, Rent, Tax  & Insurance',
    'Property taxes': 'Housing, Rent, Tax  & Insurance',
    'Rent': 'Housing, Rent, Tax  & Insurance',
    'Rented accommodation': 'Housing, Rent, Tax  & Insurance',
    'Shelter': 'Housing, Rent, Tax  & Insurance',
    'Shelter (1986)': 'Housing, Rent, Tax  & Insurance',
    'Tenants\' maintenance, repairs and other expenses': 'Housing, Rent, Tax  & Insurance'

}



In [14]:
grocery_dataframe['Category'] = grocery_dataframe['Products and product groups'].map(dict_grocery)

grocery_dataframe.drop(columns = ['Products and product groups'], inplace=True)
print(grocery_dataframe.head())
grocery_dataframe_groupby = grocery_dataframe.groupby(['REF_DATE', 'Category']).mean().reset_index()

       REF_DATE  VALUE Category
670  2017-01-01  132.8    Fruit
671  2017-02-01  133.9    Fruit
672  2017-03-01  133.5    Fruit
673  2017-04-01  136.6    Fruit
674  2017-05-01  139.0    Fruit


/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/1689205903.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grocery_dataframe['Category'] = grocery_dataframe['Products and product groups'].map(dict_grocery)
/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/1689205903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grocery_dataframe.drop(columns = ['Products and product groups'], inplace=True)


In [15]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Fish and Seafood', 'Dairy')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [16]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Grains', 'Pasta', 'Vegetables')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [17]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Sweets', 'Fruit')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [18]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'Meat', 'Pet Supplies')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [19]:
plot_for_parallel_trends(grocery_dataframe_groupby, '2018-06-01', '2018-09-01', 'General Category', 'Dining Out')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [20]:
#It seems most items follow a sharp downward trend in august when china tariffs were implemented and were not as affected by Canadian tariffs
#Only dairy and seafood and fish seem to follow the parallel trends assumption 
#let us investigate differences in differences to get an idea of the impact of the tariffs on groceries that followed the aforementioned pattern
#these groceries seem to have to do with everyday food group except carbohydrates and vegetables
differences_differences(grocery_dataframe_groupby, 'Fish and Seafood', 'Dairy', '2018-06-01', '2018-09-01', '2018-08-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.477
Method:                 Least Squares   F-statistic:                     3.131
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.150
Time:                        19:35:53   Log-Likelihood:                -6.5179
No. Observations:                   8   AIC:                             21.04
Df Residuals:                       4   BIC:                             21.35
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.4852      0.446     -1.087      0.338      -1.724       0.754
tariff_non_tariffed          0.9117      0.631      1.445      0.222      -0.840       2.664
post                         2.0470      0.892      2.294      0.084      -0.431       4.525
tariff_non_tariffed:post    -3.8589      1.262     -3.058      0.038      -7.363      -0.355
==============================================================================
Omnibus:                        0.173   Durbin-Watson:                   0.955
Prob(Omnibus):                  0.917   Jarque-Bera (JB):                0.344
Skew:                           0.188   Prob(JB):                        0.842
Kurtosis:                       2.057   Cond. No.                         6.50
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
differences_differences(grocery_dataframe, 'Fish and Seafood', 'Dairy', '2018-06-01', '2018-09-01', '2018-08-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.066
Method:                 Least Squares   F-statistic:                   0.02335
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.995
Time:                        19:35:53   Log-Likelihood:                -68.071
No. Observations:                  48   AIC:                             144.1
Df Residuals:                      44   BIC:                             151.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.0231      0.228     -0.101      0.920      -0.482       0.436
tariff_non_tariffed          0.0430      0.353      0.122      0.904      -0.668       0.754
post                         0.0965      0.455      0.212      0.833      -0.821       1.014
tariff_non_tariffed:post    -0.1819      0.706     -0.258      0.798      -1.604       1.240
==============================================================================
Omnibus:                        0.255   Durbin-Watson:                   0.313
Prob(Omnibus):                  0.881   Jarque-Bera (JB):                0.420
Skew:                          -0.136   Prob(JB):                        0.811
Kurtosis:                       2.631   Cond. No.                         6.13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
#It seems the groupby shows a significant difference but when looking at the individual items, the difference is not significant
#perhaps there was not enough data and the set was underpowered but it provides some evidence from the plots that the tariffs had a slight impact on goods - let us investigate further

In [23]:
df_sales = pd.read_csv("../data/processed/Canada_Sales_Processed.csv")

In [24]:
df_sales.columns = ['PrincipleStats', 'GoodType', 'REF_DATE', 'VALUE']
df_sales['PrincipleStats'].unique()

array(['Finished goods manufactured, estimated values at end of  month',
       'Goods or work in process, estimated values at end of month',
       'New orders, estimated values of orders received during month',
       'Raw materials, fuel, supplies, components, estimated values at end of month',
       'Sales of goods manufactured (shipments)',
       'Total inventory, estimated values of total inventory at end of the month',
       'Unfilled orders, estimated values of orders at end of month'],
      dtype=object)

In [25]:
def plot_supply_and_demand_canada(sales_df, product, date1 = '2018-05-01', date2 = '2018-09-01'):
    df_good = df_sales[(df_sales['GoodType'] == product) & ((df_sales['PrincipleStats'] == 'Total inventory, estimated values of total inventory at end of the month') | (df_sales['PrincipleStats'] == 'Unfilled orders, estimated values of orders at end of month'))].copy()
    scaler = StandardScaler()
    scaler2 = StandardScaler()
    mask1 = df_good['PrincipleStats']=='Unfilled orders, estimated values of orders at end of month'
    mask2 = df_good['PrincipleStats']=='Total inventory, estimated values of total inventory at end of the month'
    df_good.loc[mask1, "VALUE"] = scaler.fit_transform(df_good.loc[mask1, 'VALUE'].values.reshape(-1, 1))
    df_good.loc[mask2, "VALUE"] = scaler2.fit_transform(df_good.loc[mask2, 'VALUE'].values.reshape(-1, 1))
    df_good = df_good[(df_good['REF_DATE']>=date1) & (df_good['REF_DATE']<=date2)]
    chart1 = alt.Chart(df_good).mark_line().encode(
    x='REF_DATE',
    y='VALUE',
    color = 'PrincipleStats',
    
    ).properties(title = product).interactive()
    return chart1

In [26]:
plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]")

alt.Chart(...)

In [27]:
#it seems more inventory meant that less orders were filled - this could be due to the tariffs - and it also seems that there were fewer unfilled orders meaning less demand for food manufacturing which means supply went down and inflation should go up.
#but since demand and inventory are similar, it is hard to say if the tariffs had an impact on inflation and the original models without the groupby may be right 

In [28]:
#lets look earlier
plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]", '2018-01-01', '2018-06-01')

alt.Chart(...)

In [29]:
model2 = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Groceries', '2018-02-01', '2018-06-01', '2018-04-01')

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

In [30]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.034
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.377
Time:                        19:35:54   Log-Likelihood:                -373.78
No. Observations:                 354   AIC:                             755.6
Df Residuals:                     350   BIC:                             771.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.5881      0.186     -3.168      0.002      -0.953      -0.223
Num_Date              -0.1264      0.117     -1.077      0.282      -0.357       0.104
treatment              0.2862      0.201      1.427      0.154      -0.108       0.681
Num_Date:treatment     0.0452      0.131      0.344      0.731      -0.213       0.304
==============================================================================
Omnibus:                       38.654   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.360
Skew:                           0.793   Prob(JB):                     1.16e-11
Kurtosis:                       3.948   Cond. No.                         14.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
plot_for_parallel_trends(df_Canada_CPI_groupby, '2018-01-01', '2018-10-01', 'Groceries', 'Education & Reading')



/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [32]:
plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-01-01', '2018-10-01', 'Groceries', 'Education & Reading')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [33]:
#The anticorrelated point seems to be Education & Reading which had the opposite response to groceries in response to the tariffs
#let us plot the chart during this time period again for further investigate invesigation

plot_supply_and_demand_canada(df_sales, "Food manufacturing [311]", "2018-02-01", "2018-12-01")

alt.Chart(...)

In [34]:
#while inventory did increase meaning a decrease in supply, there was still enough demand and that is why groceries were not impacted despite the sharp increase in April 2025.

#trudeau's tariffs appeared to decrease total inventory and restore more Canadian supply to the Canadian market
plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2017-09-01', '2018-09-01', 'Groceries', 'Services', category_3 = None)

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [35]:
#let us look at what happened in American groceries due to the tariffs and compare it to Canada

plot_structure(df_USA_CPI_groupby, 'Groceries', '2017-11-01', '2018-09-01')
model_USA_RC_grocery_cutoff_1 = regression_discontinuity_model(df_USA_CPI_Scaled, 'Groceries', '2018-02-01', '2018-04-01', '2018-03-01')
model_USA_RC_grocery_cutoff_2 = regression_discontinuity_model(df_USA_CPI_Scaled, 'Groceries', '2018-03-01', '2018-05-01', '2018-04-01')

print(model_USA_RC_grocery_cutoff_1.summary())
model_USA_RC_grocery_cutoff_2.summary()

Product:  Groceries


alt.Chart(...)

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

                            OLS Regression Results                            
Dep. Variable:                  VALUE   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.5573
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.574
Time:                        19:35:54   Log-Likelihood:                -119.45
No. Observations:                 116   AIC:                             244.9
Df Residuals:                     113   BIC:                             253.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.3113      0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.5170
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.598
Time:                        19:35:54   Log-Likelihood:                -114.25
No. Observations:                 116   AIC:                             234.5
Df Residuals:                     113   BIC:                             242.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.3601      0.059     -6.112      0.000      -0.477      -0.243
Num_Date               0.1462      0.059      2.482      0.015       0.030       0.263
treatment              0.0049      0.120      0.040      0.968      -0.234       0.244
Num_Date:treatment    -0.2187      0.142     -1.539      0.126      -0.500       0.063
==============================================================================
Omnibus:                       15.207   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.802
Skew:                           0.864   Prob(JB):                     0.000225
Kurtosis:                       3.698   Cond. No.                     4.84e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.7e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [36]:
print(df_USA_CPI_groupby['Category'].unique())
plot_for_parallel_trends(df_USA_CPI_groupby, '2017-06-01', '2018-12-01', 'Groceries', 'Education & Reading', category_3 = None)

['Alcohol & Tobacco' 'Appliances & Equipment' 'Clothing & Footwear'
 'Communication' 'Education & Reading' 'Energy' 'Furniture & Home Decor'
 'Groceries' 'Health & Personal Care' 'Household Supplies'
 'Housing & Utilities' 'Housing, Rent, Tax & Insurance'
 'Recreation & Entertainment' 'Services' 'Transportation' 'Uncategorized'
 'Utilities' 'Vehicles']


/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [37]:
plot_for_parallel_trends(df_USA_CPI_groupby, '2017-06-01', '2018-12-01', 'Groceries', 'Household Supplies', category_3 = None)

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [38]:
differences_differences(df_USA_CPI_groupby, 'Groceries', 'Education & Reading', '2018-03-01', '2018-05-01', '2018-04-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     4.176
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.199
Time:                        19:35:54   Log-Likelihood:                -2.5649
No. Observations:                   6   AIC:                             13.13
Df Residuals:                       2   BIC:                             12.30
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.1668      0.454     -2.568      0.124      -3.122       0.788
tariff_non_tariffed          2.2703      0.643      3.533      0.072      -0.495       5.035
post                         1.2923      0.787      1.642      0.242      -2.094       4.679
tariff_non_tariffed:post    -2.3945      1.113     -2.151      0.164      -7.184       2.395
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.353
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.459
Skew:                          -0.000   Prob(JB):                        0.795
Kurtosis:                       1.646   Cond. No.                         6.32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
#Trump tariffs seemed not to have a significant impact on groceries as compared to education & reading which did not get tariffed - let us look at a similar trend to groceries - Household Supplies

In [40]:
model_USA_RC_HS_cutoff = regression_discontinuity_model(df_USA_CPI_Scaled, 'Household Supplies', '2018-03-01', '2018-05-01', '2018-04-01')
model_USA_RC_HS_cutoff.summary()

Product:  Household Supplies


alt.Chart(...)

alt.Chart(...)

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     8.832
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.102
Time:                        19:35:54   Log-Likelihood:                 6.9448
No. Observations:                   5   AIC:                            -7.890
Df Residuals:                       2   BIC:                            -9.061
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.6627      0.041    -16.043      0.004      -0.840      -0.485
Num_Date               0.2837      0.041      6.867      0.021       0.106       0.461
treatment              0.1171      0.079      1.481      0.277      -0.223       0.457
Num_Date:treatment    -0.4961      0.104     -4.773      0.041      -0.943      -0.049
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.538
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.054
Skew:                          -0.000   Prob(JB):                        0.974
Kurtosis:                       2.493   Cond. No.                     3.73e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.37e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [41]:
plot_for_parallel_trends(df_USA_CPI_groupby, '2017-06-01', '2018-12-01', 'Household Supplies', 'Education & Reading')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [42]:
#The small impact on the visual did not seem to reach significant difference 

differences_differences(df_USA_CPI_groupby, 'Household Supplies', 'Education & Reading', '2018-01-01', '2018-08-01', '2018-03-01', '2018-06-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     9.928
Date:                Wed, 12 Mar 2025   Prob (F-statistic):            0.00964
Time:                        19:35:54   Log-Likelihood:                -5.2605
No. Observations:                  10   AIC:                             18.52
Df Residuals:                       6   BIC:                             19.73
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.2018      0.305     -0.661      0.533      -0.949       0.545
tariff_non_tariffed         -0.6157      0.432     -1.427      0.204      -1.672       0.440
post                         1.9485      0.483      4.038      0.007       0.768       3.129
tariff_non_tariffed:post    -1.3485      0.682     -1.976      0.096      -3.018       0.321
==============================================================================
Omnibus:                        0.855   Durbin-Watson:                   2.080
Prob(Omnibus):                  0.652   Jarque-Bera (JB):                0.623
Skew:                           0.521   Prob(JB):                        0.732
Kurtosis:                       2.359   Cond. No.                         6.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
#The impact isnt seen with Differences in Differences but may be seen with regression discontinuity
model_may_groceries_USA = regression_discontinuity_model(df_USA_CPI_groupby, 'Groceries', '2018-03-01', '2018-09-01', '2018-05-01')
model_may_groceries_USA.summary()

Product:  Groceries


alt.Chart(...)

alt.Chart(...)

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     83.02
Date:                Wed, 12 Mar 2025   Prob (F-statistic):             0.0119
Time:                        19:35:54   Log-Likelihood:                 21.612
No. Observations:                   6   AIC:                            -35.22
Df Residuals:                       2   BIC:                            -36.06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.4697      0.026    -18.380      0.003      -0.580      -0.360
Num_Date               0.0871      0.016      5.391      0.033       0.018       0.157
treatment             -0.2312      0.027     -8.472      0.014      -0.349      -0.114
Num_Date:treatment    -0.0116      0.017     -0.685      0.564      -0.085       0.061
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.981
Prob(Omnibus):                    nan   Jarque-Bera (JB):                1.130
Skew:                          -1.055   Prob(JB):                        0.568
Kurtosis:                       3.254   Cond. No.                         22.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
#The effect of the news on trade talks with China on groceries seems small but looking at the news of intellectual property on groceries will give us a bigger picture of the impact of the overall consequences of the breakdown

model_USA_RC_grocery_IP = regression_discontinuity_model(df_USA_CPI_Scaled, 'Utilities', '2017-08-01', '2018-02-01', '2017-12-01')

Product:  Utilities


alt.Chart(...)

alt.Chart(...)

In [45]:
model_USA_RC_grocery_IP.summary()

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                     38.21
Date:                Wed, 12 Mar 2025   Prob (F-statistic):             0.0256
Time:                        19:35:54   Log-Likelihood:                 6.7578
No. Observations:                   6   AIC:                            -5.516
Df Residuals:                       2   BIC:                            -6.349
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.0006      0.166      0.004      0.997      -0.715       0.717
Num_Date              -0.1320      0.061     -2.172      0.162      -0.393       0.130
treatment              0.6514      0.215      3.032      0.094      -0.273       1.576
Num_Date:treatment    -1.6537      0.202     -8.205      0.015      -2.521      -0.786
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.693
Prob(Omnibus):                    nan   Jarque-Bera (JB):                1.002
Skew:                          -0.996   Prob(JB):                        0.606
Kurtosis:                       3.197   Cond. No.                         11.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
#lets switch our attention to clothing and footwear which is a category that could be affected by tariffs
model_clothing = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Clothing & Footwear', '2018-05-01', '2018-09-01', '2018-07-01')


Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

In [47]:
model_clothing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.297
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     10.54
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           7.32e-06
Time:                        19:35:54   Log-Likelihood:                -70.087
No. Observations:                  79   AIC:                             148.2
Df Residuals:                      75   BIC:                             157.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.8510      0.337     -2.524      0.014      -1.523      -0.179
Num_Date              -0.4773      0.213     -2.239      0.028      -0.902      -0.053
treatment              0.0981      0.364      0.270      0.788      -0.627       0.824
Num_Date:treatment     1.0331      0.239      4.320      0.000       0.557       1.510
==============================================================================
Omnibus:                        2.844   Durbin-Watson:                   2.278
Prob(Omnibus):                  0.241   Jarque-Bera (JB):                2.261
Skew:                          -0.404   Prob(JB):                        0.323
Kurtosis:                       3.186   Cond. No.                         14.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-05-01', '2018-09-01', 'Clothing & Footwear', 'Utilities')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [49]:
differences_differences(df_Canada_CPI_NonScaled, 'Clothing & Footwear', 'Utilities', '2018-05-01', '2018-09-01', '2018-07-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.414
Method:                 Least Squares   F-statistic:                     20.80
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           4.38e-10
Time:                        19:35:54   Log-Likelihood:                -96.338
No. Observations:                  85   AIC:                             200.7
Df Residuals:                      81   BIC:                             210.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    2.6371      0.445      5.932      0.000       1.753       3.522
tariff_non_tariffed         -2.8127      0.458     -6.139      0.000      -3.724      -1.901
post                         0.0022      0.703      0.003      0.998      -1.396       1.401
tariff_non_tariffed:post     0.0247      0.724      0.034      0.973      -1.417       1.466
==============================================================================
Omnibus:                       27.379   Durbin-Watson:                   0.428
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.481
Skew:                           1.489   Prob(JB):                     2.67e-09
Kurtosis:                       4.510   Cond. No.                         19.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
differences_differences(df_Canada_CPI_groupby_NonScaled, 'Clothing & Footwear', 'Utilities', '2018-05-01', '2018-09-01', '2018-07-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.353e+04
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           7.06e-12
Time:                        19:35:54   Log-Likelihood:                 29.908
No. Observations:                  10   AIC:                            -51.82
Df Residuals:                       6   BIC:                            -50.61
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.9994      0.009    110.282      0.000       0.977       1.022
tariff_non_tariffed         -1.9999      0.013   -156.056      0.000      -2.031      -1.969
post                         0.0014      0.014      0.097      0.926      -0.034       0.036
tariff_non_tariffed:post     0.0002      0.020      0.011      0.991      -0.049       0.050
==============================================================================
Omnibus:                        0.865   Durbin-Watson:                   1.670
Prob(Omnibus):                  0.649   Jarque-Bera (JB):                0.404
Skew:                           0.459   Prob(JB):                        0.817
Kurtosis:                       2.640   Cond. No.                         6.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
plot_supply_and_demand_canada(df_sales, "Leather and allied product manufacturing [316]", "2018-01-01", "2018-09-01")

alt.Chart(...)

In [52]:

plot_supply_and_demand_canada(df_sales, "Apparel manufacturing [315]", "2018-01-01", "2018-09-01")

alt.Chart(...)

In [53]:
#IT seems like manufacturing demand for clothing took a hit during July which means there were less orders from clothing stores and supply for clothing was decreased

#While there seems to be no difference in the differences and differences model (perhaps it did not completely follow the parallel trends assumption), the regression discontinuity model shows a significant difference in inflation for clothing and footwear during the time period of the tariffs

In [54]:
model_clothing_US_tariff = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Clothing & Footwear', '2018-02-01', '2018-06-01', '2018-04-01')



Product:  Clothing & Footwear


alt.Chart(...)

alt.Chart(...)

In [55]:
model_clothing_US_tariff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     6.824
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           0.000397
Time:                        19:35:54   Log-Likelihood:                -83.106
No. Observations:                  79   AIC:                             174.2
Df Residuals:                      75   BIC:                             183.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.6005      0.398      1.511      0.135      -0.191       1.392
Num_Date               0.5662      0.251      2.252      0.027       0.065       1.067
treatment             -0.1030      0.429     -0.240      0.811      -0.958       0.752
Num_Date:treatment    -0.9990      0.282     -3.543      0.001      -1.561      -0.437
==============================================================================
Omnibus:                        9.775   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.008   Jarque-Bera (JB):                9.578
Skew:                          -0.798   Prob(JB):                      0.00832
Kurtosis:                       3.604   Cond. No.                         14.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [56]:
plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-02-01', '2018-06-01', 'Clothing & Footwear', 'Utilities')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [57]:
differences_differences(df_Canada_CPI_NonScaled, 'Clothing & Footwear', 'Utilities', '2018-02-01', '2018-06-01', '2018-04-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.410
Method:                 Least Squares   F-statistic:                     20.44
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           5.89e-10
Time:                        19:35:54   Log-Likelihood:                -96.654
No. Observations:                  85   AIC:                             201.3
Df Residuals:                      81   BIC:                             211.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    2.6175      0.446      5.866      0.000       1.730       3.505
tariff_non_tariffed         -2.7765      0.460     -6.037      0.000      -3.692      -1.861
post                         0.0201      0.705      0.028      0.977      -1.384       1.424
tariff_non_tariffed:post    -0.0329      0.727     -0.045      0.964      -1.480       1.414
==============================================================================
Omnibus:                       29.777   Durbin-Watson:                   0.436
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               45.433
Skew:                           1.557   Prob(JB):                     1.36e-10
Kurtosis:                       4.770   Cond. No.                         19.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
differences_differences(df_Canada_CPI_groupby_NonScaled, 'Clothing & Footwear', 'Utilities', '2018-02-01', '2018-06-01', '2018-04-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.288e+04
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           8.18e-12
Time:                        19:35:54   Log-Likelihood:                 29.663
No. Observations:                  10   AIC:                            -51.33
Df Residuals:                       6   BIC:                            -50.12
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.9947      0.009    107.110      0.000       0.972       1.017
tariff_non_tariffed         -1.9957      0.013   -151.953      0.000      -2.028      -1.964
post                         0.0130      0.015      0.888      0.409      -0.023       0.049
tariff_non_tariffed:post    -0.0104      0.021     -0.501      0.634      -0.061       0.040
==============================================================================
Omnibus:                        0.307   Durbin-Watson:                   1.153
Prob(Omnibus):                  0.858   Jarque-Bera (JB):                0.429
Skew:                          -0.104   Prob(JB):                        0.807
Kurtosis:                       2.007   Cond. No.                         6.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
#It seems trumps tariffs had little impact on clothing and footwear inflation in Canada and if anything it decreased inflation for clothing and footwear for Canadian goods.

In [60]:
plot_supply_and_demand_canada(df_sales, "Apparel manufacturing [315]", "2018-02-01", "2018-09-01")

alt.Chart(...)

In [61]:
#It appears to have caused a decrease in orders or manufacturing demand which means less supply for clothing and footwear, however, demand was still above supply in this time frame - the impact may not have been felt until in July/August

#This may mean the inflation felt by the tariffs seen earlier were actually due to trump's tariffs rather than Canadian tariffs

In [62]:
#let us now look at transportation and see if the tariffs had an impact on transportation

In [63]:
model_transportation_Canada_tariff = regression_discontinuity_model(df_Canada_CPI_Scaled, 'Transportation', '2018-05-01', '2018-09-01', '2018-07-01')


Product:  Transportation


alt.Chart(...)

alt.Chart(...)

In [64]:
model_transportation_Canada_tariff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     4.770
Date:                Wed, 12 Mar 2025   Prob (F-statistic):            0.00403
Time:                        19:35:54   Log-Likelihood:                -76.841
No. Observations:                  89   AIC:                             161.7
Df Residuals:                      85   BIC:                             171.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.3575      0.309      1.155      0.251      -0.258       0.973
Num_Date               0.2609      0.196      1.333      0.186      -0.128       0.650
treatment              0.1744      0.334      0.522      0.603      -0.490       0.839
Num_Date:treatment    -0.4829      0.219     -2.201      0.030      -0.919      -0.047
==============================================================================
Omnibus:                        6.129   Durbin-Watson:                   1.830
Prob(Omnibus):                  0.047   Jarque-Bera (JB):                4.431
Skew:                           0.412   Prob(JB):                        0.109
Kurtosis:                       2.281   Cond. No.                         14.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
#Initially it seems Canadian tariffs decreased inflation for transportation but let us look further

In [66]:
plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-02-01', '2018-11-01', 'Transportation', 'Energy')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [67]:
#it appears that trumps tariffs in March/April had a significant influence on energy which may have caused transportation to rise in May
#trudeau's tariffs in July may have caused a decrease in transportation inflation
#let us see if we can do a differences in differences with housing, rent, tax and insurance which were largely unaffected by the tariffs

plot_for_parallel_trends(df_Canada_CPI_groupby_NonScaled, '2018-01-01', '2018-11-01', 'Transportation', 'Housing, Rent, Tax  & Insurance')

/var/folders/t8/6bysc4l55cn05grw21ffck1w0000gn/T/ipykernel_86693/2805251915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'] = scale.fit_transform(df_Canada_CPI_Scaled_US_on_Canada_Tariffs_1['VALUE'].values.reshape(-1, 1))


alt.Chart(...)

In [68]:
differences_differences(df_Canada_CPI_groupby_NonScaled, 'Transportation', 'Housing, Rent, Tax  & Insurance', '2018-03-01', '2018-07-01', '2018-05-01')

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     68.60
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           4.92e-05
Time:                        19:35:54   Log-Likelihood:                 3.6297
No. Observations:                  10   AIC:                            0.7405
Df Residuals:                       6   BIC:                             1.951
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.9995      0.125     -7.967      0.000      -1.307      -0.693
tariff_non_tariffed          1.5107      0.177      8.515      0.000       1.077       1.945
post                         0.2041      0.198      1.029      0.343      -0.281       0.689
tariff_non_tariffed:post     0.8129      0.281      2.898      0.027       0.126       1.499
==============================================================================
Omnibus:                        5.853   Durbin-Watson:                   2.316
Prob(Omnibus):                  0.054   Jarque-Bera (JB):                2.092
Skew:                           1.039   Prob(JB):                        0.351
Kurtosis:                       3.836   Cond. No.                         6.41
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [69]:
differences_differences(df_Canada_CPI_NonScaled, 'Transportation', 'Housing, Rent, Tax  & Insurance', '2018-03-01', '2018-07-01', '2018-05-01')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.3729
Date:                Wed, 12 Mar 2025   Prob (F-statistic):              0.773
Time:                        19:35:54   Log-Likelihood:                -198.08
No. Observations:                 140   AIC:                             404.2
Df Residuals:                     136   BIC:                             415.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -0.0841      0.184     -0.456      0.649      -0.449       0.281
tariff_non_tariffed          0.0504      0.230      0.219      0.827      -0.405       0.505
post                         0.0308      0.292      0.106      0.916      -0.546       0.608
tariff_non_tariffed:post     0.1530      0.364      0.421      0.675      -0.566       0.872
==============================================================================
Omnibus:                       34.102   Durbin-Watson:                   0.315
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               70.962
Skew:                           1.044   Prob(JB):                     3.90e-16
Kurtosis:                       5.794   Cond. No.                         7.61
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [70]:
#Despite the visualized increase in transportation costs in May, the differences in differences model shows no significant difference in inflation between transportation and housing, rent, tax and insurance

#This may be due to a violation of the parallel trends assumption since Canadian taxes were implemented in July affected the post period. There seems no good comparator for transportation so we must rely on visualizations to see the impact of the tariffs on transportation
#let us plot how equipment manufacturing was affected by the tariffs

plot_supply_and_demand_canada(df_sales, "Transportation equipment manufacturing [336]", "2018-01-01", "2018-11-01")

alt.Chart(...)

In [71]:
#It seems trumps tariffs in March/April did not change Total Inventory production until May which may have been a composition of factors such as the global climate 
# where demand for transportation equipment was higher and supply was low is where the inflation occurred
#This means that consumer demand increased or stayed the same for transportation equipment and supply was low which caused inflation to rise in May

#we can quantify this effect using regression discontinuity

model_sales_RC_transportation = regression_discontinuity_model(df_sales, 'Total inventory, estimated values of total inventory at end of the month',
       '2018-01-01', '2018-09-01', '2018-07-01', feature = 'PrincipleStats', goodtype = 'Transportation equipment manufacturing [336]')

Product:  Total inventory, estimated values of total inventory at end of the month


alt.Chart(...)

alt.Chart(...)

KeyError: 'Category'

In [67]:
model_sales_RC_transportation.summary()

/Users/richardwcheng/Library/Python/3.9/lib/python/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     15.30
Date:                Wed, 12 Mar 2025   Prob (F-statistic):             0.0117
Time:                        17:48:49   Log-Likelihood:                -107.02
No. Observations:                   8   AIC:                             222.0
Df Residuals:                       4   BIC:                             222.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           1.602e+07   2.05e+05     77.949      0.000    1.54e+07    1.66e+07
Num_Date            3.422e+05   5.28e+04      6.487      0.003    1.96e+05    4.89e+05
treatment          -7.985e+05   3.02e+05     -2.648      0.057   -1.64e+06    3.87e+04
Num_Date:treatment -4.455e+05   3.17e+05     -1.408      0.232   -1.32e+06    4.33e+05
==============================================================================
Omnibus:                        0.387   Durbin-Watson:                   1.788
Prob(Omnibus):                  0.824   Jarque-Bera (JB):                0.197
Skew:                          -0.287   Prob(JB):                        0.906
Kurtosis:                       2.488   Cond. No.                         16.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""